## Attribution automatique avant d'utiliser le modèle

On attribut directement les lignes de Soil_Type 37 au Cover_Type 7.

In [1]:
import pandas as pd
import scipy.stats as stats
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

data = pd.read_csv('train.csv')

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# On sélectionne les colonnes d'entrée (features) et la cible
X = data.drop(columns=['Cover_Type'])
y = data['Cover_Type']

# On sélectionne les colonnes d'entrée (features) et la cible
X = data.drop(columns=['Cover_Type'])
y = data['Cover_Type']

# On divise les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# On normalise les données continues 
from sklearn.preprocessing import StandardScaler

continuous_columns = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 
                      'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 
                      'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']
scaler = StandardScaler()

# On applique le scaler uniquement sur les colonnes continues
X_train[continuous_columns] = scaler.fit_transform(X_train[continuous_columns])
X_test[continuous_columns] = scaler.transform(X_test[continuous_columns])

# On ajuste les labels pour qu'ils commencent à 0 pour XGBoost
y_train_adj = y_train - 1
y_test_adj = y_test - 1

In [3]:
# Étape 1 : Attribuer Cover_Type 7 aux lignes avec Soil_Type37
def assign_soil_type_37_to_cover_7(df):
    # Créer une copie pour éviter de modifier le DataFrame original
    df = df.copy()
    
    # Créer une colonne `Predicted_Cover_Type` pour stocker les attributions directes
    df['Predicted_Cover_Type'] = None
    
    # Attribuer Cover_Type 7 aux lignes avec Soil_Type37 activé
    df.loc[df['Soil_Type37'] == 1, 'Predicted_Cover_Type'] = 7
    
    return df

# Appliquer la règle aux ensembles d'entraînement et de test
X_train_with_auto_labels = assign_soil_type_37_to_cover_7(X_train)
X_test_with_auto_labels = assign_soil_type_37_to_cover_7(X_test)

# Étape 2 : Séparer les lignes avec Cover_Type déterminé et celles qui nécessitent un modèle
# Lignes déterministes (Soil_Type37 -> Cover_Type 7)
train_deterministic = X_train_with_auto_labels.dropna(subset=['Predicted_Cover_Type'])
test_deterministic = X_test_with_auto_labels.dropna(subset=['Predicted_Cover_Type'])

# Lignes sans Cover_Type déterminé (passer au modèle)
train_to_predict = X_train_with_auto_labels[X_train_with_auto_labels['Predicted_Cover_Type'].isna()]
test_to_predict = X_test_with_auto_labels[X_test_with_auto_labels['Predicted_Cover_Type'].isna()]

# Extraire les labels pour les lignes déterminées
y_train_deterministic = train_deterministic['Predicted_Cover_Type']
y_test_deterministic = test_deterministic['Predicted_Cover_Type']

# Pour les lignes restantes, enlevez la colonne temporaire `Predicted_Cover_Type`
X_train_to_predict = train_to_predict.drop(columns=['Predicted_Cover_Type'])
X_test_to_predict = test_to_predict.drop(columns=['Predicted_Cover_Type'])
y_train_to_predict = y_train[train_to_predict.index]  # Labels d'origine pour entraînement


On va prédire le reste avec le modèle de stacking.

In [ ]:
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Définir les modèles de base avec les meilleurs hyperparamètres
random_forest = RandomForestClassifier(
    n_estimators=300,  
    max_depth=None,      
    random_state=42,
    max_features='sqrt',
    min_samples_split=2,
    min_samples_leaf=1,
)

xgboost = XGBClassifier(
    colsample_bytree=0.928,
    gamma=0.049,
    learning_rate=0.241,
    max_depth=9,
    n_estimators=300,
    subsample=0.983,
    objective='multi:softmax',
    num_class=7,
    random_state=42
)

lightgbm = LGBMClassifier(
    colsample_bytree=0.949,
    learning_rate=0.092,
    max_depth=15,
    n_estimators=300,
    num_leaves=100,
    subsample=0.519,
    objective='multiclass',
    num_class=7,
    random_state=42,
    verbosity=-1
)

# Définir le méta-modèle (un modèle simple comme la régression logistique)
meta_model = LogisticRegression(max_iter=1000)

# Créer le StackingClassifier
stacking_model = StackingClassifier(
    estimators=[
        ('rf', random_forest),
        ('xgb', xgboost),
        ('lgbm', lightgbm)
    ],
    final_estimator=meta_model,
    cv=5
)

y_train_to_predict_adj = y_train_to_predict - 1

# Entraîner le modèle de stacking sur les lignes non déterministes
stacking_model.fit(X_train_to_predict, y_train_to_predict_adj)

# Prédire sur les lignes non déterministes de l'ensemble de test
y_pred_stack = stacking_model.predict(X_test_to_predict)

y_pred_stack = y_pred_stack + 1

# Assigner les prédictions de `y_pred_stack` aux lignes non déterministes dans `y_test_adj`
test_to_predict['Predicted_Cover_Type'] = y_pred_stack

# Fusionner les prédictions des lignes déterministes et non déterministes
y_test_final = pd.concat([
    test_deterministic[['Predicted_Cover_Type']],
    test_to_predict[['Predicted_Cover_Type']]
]).sort_index()

# Conversion explicite en int, et gestion des NaN (en s'assurant qu'il n'y en a pas)
y_test_final = y_test_final['Predicted_Cover_Type'].astype(int)
y_test = y_test.astype(int)  # Assurez-vous que y_test_adj est aussi en int


/var/folders/24/f9mpl0gx1rs5tb1xb_z_cqmw0000gn/T/ipykernel_88296/2464779910.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_to_predict['Predicted_Cover_Type'] = y_pred_stack


In [ ]:
# Réindexer y_test_final pour qu'il corresponde à y_test_adj
y_test_final = y_test_final.reindex(y_test.index)

# Vérifiez que les index sont bien alignés
if y_test_final.index.equals(y_test.index):
    print("Les index sont bien alignés.")
else:
    print("Les index ne sont pas alignés. Vérifiez les données.")

# Calculer la précision et le rapport de classification
from sklearn.metrics import accuracy_score, classification_report

accuracy_stack = accuracy_score(y_test, y_test_final)
print("Précision du modèle de stacking :", accuracy_stack)
print("Rapport de classification du modèle de stacking :\n", classification_report(y_test, y_test_final))


Les index sont bien alignés.
Précision du modèle de stacking : 0.892636684303351
Rapport de classification du modèle de stacking :
               precision    recall  f1-score   support

           1       0.80      0.81      0.81       648
           2       0.80      0.74      0.77       648
           3       0.88      0.90      0.89       648
           4       0.96      0.98      0.97       648
           5       0.93      0.95      0.94       648
           6       0.90      0.92      0.91       648
           7       0.96      0.95      0.95       648

    accuracy                           0.89      4536
   macro avg       0.89      0.89      0.89      4536
weighted avg       0.89      0.89      0.89      4536



C'est exactement le même résultat que le modèle de stacking, on remarque que notre modèle de stacking attribut déjà les Soil_type 37 au Cover_Type 7.

On va voir ce que cela donne sur le test complet.

In [20]:
X_test_full = pd.read_csv('test-full.csv')

In [21]:
# Appliquer la règle aux ensembles d'entraînement et de test
X_with_auto_labels = assign_soil_type_37_to_cover_7(X)
X_test_full_with_auto_labels = assign_soil_type_37_to_cover_7(X_test_full)

# Étape 2 : Séparer les lignes avec Cover_Type déterminé et celles qui nécessitent un modèle
# Lignes déterministes (Soil_Type37 -> Cover_Type 7)
try_deterministic = X_train_with_auto_labels.dropna(subset=['Predicted_Cover_Type'])
test_deterministic = X_test_with_auto_labels.dropna(subset=['Predicted_Cover_Type'])

# Lignes sans Cover_Type déterminé (passer au modèle)
train_to_predict = X_train_with_auto_labels[X_train_with_auto_labels['Predicted_Cover_Type'].isna()]
test_to_predict = X_test_with_auto_labels[X_test_with_auto_labels['Predicted_Cover_Type'].isna()]

# Extraire les labels pour les lignes déterminées
y_train_deterministic = train_deterministic['Predicted_Cover_Type']
y_test_deterministic = test_deterministic['Predicted_Cover_Type']

# Pour les lignes restantes, enlevez la colonne temporaire `Predicted_Cover_Type`
X_train_to_predict = train_to_predict.drop(columns=['Predicted_Cover_Type'])
X_test_to_predict = test_to_predict.drop(columns=['Predicted_Cover_Type'])
y_train_to_predict = y[train_to_predict.index]  # Labels d'origine pour entraînement

In [ ]:
y_train_to_predict_adj = y_train_to_predict - 1

# Entraîner le modèle de stacking sur les lignes non déterministes
stacking_model.fit(X_train_to_predict, y_train_to_predict_adj)

# Initialiser une colonne pour les prédictions avec NaN
y_test_final = pd.DataFrame(index=X_test_full.index, columns=['Predicted_Cover_Type'])

# Appliquer la règle : affecter Cover_Type 7 aux lignes où Soil_Type37 == 1
mask_soil_type_37 = X_test_full['Soil_Type37'] == 1
y_test_final.loc[mask_soil_type_37, 'Predicted_Cover_Type'] = 7

# Extraire les lignes qui nécessitent une prédiction
X_test_to_predict = X_test_full[y_test_final['Predicted_Cover_Type'].isna()]

# Prédire les Cover_Type pour ces lignes avec le modèle de stacking
# stacking_model est supposé être le modèle de stacking que vous avez déjà entraîné
y_pred_model = stacking_model.predict(X_test_to_predict)

# Insérer les prédictions du modèle dans y_test_final pour les lignes non déterministes
y_test_final.loc[X_test_to_predict.index, 'Predicted_Cover_Type'] = y_pred_model

# Conversion de y_test_final en type entier si nécessaire
y_test_final['Predicted_Cover_Type'] = y_test_final['Predicted_Cover_Type'].astype(int)

In [67]:
y_test_final = y_test_final +1

In [71]:
# Assurez-vous que les prédictions sont alignées sur l'index de X_test_full
y_test_final = y_test_final.reindex(X_test_full.index)

# Créer le DataFrame de soumission avec les Id et les prédictions finales
submission_df = pd.DataFrame({
    'Id': X_test_full.index +1,  # Utilisez l'index de X_test_full comme 'Id'
    'Cover_Type': y_test_final['Predicted_Cover_Type']
})

# Sauvegarder le DataFrame en fichier CSV sans inclure l'index
submission_df.to_csv('soumissions/submission_final.csv', index=False)

print("Fichier de soumission créé avec succès : soumissions/submission_final.csv")

Fichier de soumission créé avec succès : soumissions/submission_final.csv


In [72]:
submission_df0 = pd.read_csv('soumissions/submission.csv')

In [73]:
print(submission_df)
print(submission_df0)


            Id  Cover_Type
0            1           7
1            2           7
2            3           1
3            4           1
4            5           7
...        ...         ...
581007  581008           7
581008  581009           7
581009  581010           7
581010  581011           7
581011  581012           7

[581012 rows x 2 columns]
            Id  Cover_Type
0            1           5
1            2           5
2            3           2
3            4           2
4            5           5
...        ...         ...
581007  581008           3
581008  581009           3
581009  581010           3
581010  581011           3
581011  581012           3

[581012 rows x 2 columns]
